<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=227582dea83ff6883ff232dde26753ce2e0a211db3e0948cabeec4874c2ee4c7
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-03 09:21:19
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  57.44 K
Current:  1.41 C
-------------------
Today PnL: 26.16 K (0.19%)
Current PnL: -21.00 L (-13.94%)
CY Booked + Current PnL: -9.56 L (-6.34%)
-------------------
Total profit:  1.38 L
Total loss:  -22.38 L
-------------------
Total Booked + Current PnL: 17.42 L (14.08%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.14%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.29 C
Est FTT PnL: 88.31 L (62.82%)
Deployed:  1.24 C
Current:  1.41 C
CAGR/XIRR %: 7.81%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.83,65.0,H-MC,12.87,114575.0,14071.0,15032.0,0.78,14.00,13.12,28.96,79.0,0.94,0.82,52.72,MH,ATH,METALS
77,TTKPRESTIG,770.00,94.45,40.0,M-SC,1.70,83781.0,-16996.0,17091.0,0.39,-16.86,20.40,0.09,245.0,-0.99,0.60,10.07,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,9.63,48.0,H-MC,2.97,189320.0,24650.0,18686.0,0.82,14.97,9.87,26.31,89.0,1.32,1.35,24.15,X40N,NTT,AC
42,ITC,452.00,-38.77,45.0,H-LC,2.18,197165.0,-2973.0,23857.0,-0.59,-1.49,12.10,10.44,4.0,-0.12,1.41,4.00,X40,NTT,FMCG
84,VOLTAS,1530.00,-2.90,45.0,H-MC,2.21,203745.0,12003.0,25753.0,0.35,6.26,12.64,19.69,99.0,0.47,1.46,13.48,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TATAMOTORS,1065.0,-15.57,64.0,M-LC,2.50,266896.0,-9627.0,119703.0,2.35,-3.48,44.85,39.81,54.0,-0.08,1.91,27.91,XY24,NTT,AUTO
28,HATHWAY,31.4,892.75,43.0,H-SC,2.61,85620.0,-23100.0,102778.0,1.71,-21.25,120.04,73.29,161.0,-0.22,0.61,14.53,XR,NTT,ENTERTAINMENT
36,IEX,219.0,-32.01,49.0,H-SC,17.62,194594.0,-5360.0,102590.0,1.57,-2.68,52.72,48.63,115.0,-0.05,1.39,8.56,XR,NTT,MISC
58,REPCOHOME,880.0,-60.51,50.0,H-SC,4.05,227362.0,-59143.0,316488.0,1.32,-20.64,139.20,89.82,134.0,-0.19,1.62,20.29,XY24,NTT,FINANCE
39,INDUSINDBK,1800.0,249.44,52.0,L-MC,12.02,42904.0,-34302.0,59697.0,1.29,-44.43,139.14,32.89,258.0,-0.57,0.31,18.15,XR,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,GREENPANEL,537.00,212.02,50.0,M-SC,2.07,139373.0,-43705.0,122690.0,-3.94,-23.87,88.03,43.14,230.0,-0.36,1.00,31.10,XY24,NTT,MISC
66,SIS,528.00,2080.83,41.0,H-SC,3.71,88013.0,-23019.0,46629.0,-1.13,-20.73,52.98,21.26,156.0,-0.49,0.63,18.91,OX40N,NTT,MISC
29,HAVELLS,2069.16,-6.46,31.0,H-MC,1.56,231691.0,-16040.0,91101.0,-1.12,-6.47,39.32,30.30,92.0,-0.18,1.65,5.96,X40,ATH,ELECTRICAL
20,COALINDIA,497.97,7.46,44.0,L-LC,4.25,142339.0,-8122.0,41905.0,-0.98,-5.40,29.44,22.46,167.0,-0.19,1.02,11.70,XY25,ATH,MINING
31,HINDUNILVR,2922.00,-10.50,41.0,H-LC,7.94,251180.0,-5883.0,41018.0,-0.96,-2.29,16.33,13.67,24.0,-0.14,1.79,17.65,XY25,NTT,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,ICICIPRULI,790.0,-21.11,48.0,H-MC,2.16,135747.0,-41.0,42787.0,0.68,-0.03,31.52,31.48,107.0,-0.00,0.97,12.08,X40,ATH,INSURANCE
86,WIPRO,420.0,-15.15,41.0,M-LC,5.77,149463.0,-1482.0,110931.0,0.71,-0.98,74.22,72.51,53.0,-0.01,1.07,4.98,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,107.70,66.0,H-SC,12.14,131085.0,-11394.0,30910.0,0.66,-8.00,23.58,13.70,163.0,-0.37,0.94,54.72,OX40N,NTT,BANKS
45,KANSAINER,340.0,-67.42,57.0,H-SC,1.82,224523.0,-45144.0,81479.0,0.10,-16.74,36.29,13.47,138.0,-0.55,1.60,13.56,XY24,NTT,PAINTS
77,TTKPRESTIG,770.0,94.45,40.0,M-SC,1.70,83781.0,-16996.0,17091.0,0.39,-16.86,20.40,0.09,245.0,-0.99,0.60,10.07,OX40N,NTT,DURABLES
18,CERA,9475.0,-22.91,36.0,H-SC,1.49,140748.0,-35155.0,77172.0,0.22,-19.99,54.83,23.89,149.0,-0.46,1.01,21.55,OX40N,NTT,CERAMICS
66,SIS,528.0,2080.83,41.0,H-SC,3.71,88013.0,-23019.0,46629.0,-1.13,-20.73,52.98,21.26,156.0,-0.49,0.63,18.91,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1244.50,69.0,M-SC,18.27,182952.0,1809.0,43652.0,-0.06,1.00,23.86,25.10,235.0,0.04,1.31,46.62,XY24,NTT,HEALTHCARE
60,SAIL,228.00,49.16,62.0,M-MC,11.95,234984.0,10022.0,157627.0,1.14,4.45,67.08,74.53,192.0,0.06,1.68,38.15,XY24,BTT,STEEL
32,HINDZINC,730.22,28.52,65.0,M-LC,4.36,209605.0,4529.0,108051.0,0.10,2.21,51.55,54.89,52.0,0.04,1.50,26.84,X5K,ATH,METALS
14,BLUESTARCO,2080.00,9.63,48.0,H-MC,2.97,189320.0,24650.0,18686.0,0.82,14.97,9.87,26.31,89.0,1.32,1.35,24.15,X40N,NTT,AC
84,VOLTAS,1530.00,-2.90,45.0,H-MC,2.21,203745.0,12003.0,25753.0,0.35,6.26,12.64,19.69,99.0,0.47,1.46,13.48,XY25,NTT,AC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.83,65.0,H-MC,12.87,114575.0,14071.0,15032.0,0.78,14.00,13.12,28.96,79.0,0.94,0.82,52.72,MH,ATH,METALS
11,BANKINDIA,190.00,-25.47,68.0,H-MC,10.05,192435.0,12627.0,99778.0,0.39,7.02,51.85,62.52,88.0,0.13,1.37,43.50,XR,NTT,BANKS
32,HINDZINC,730.22,28.52,65.0,M-LC,4.36,209605.0,4529.0,108051.0,0.10,2.21,51.55,54.89,52.0,0.04,1.50,26.84,X5K,ATH,METALS
37,INDIAMART,4850.92,-54.45,37.0,H-SC,9.41,127126.0,3790.0,129974.0,-0.01,3.07,102.24,108.45,119.0,0.03,0.91,26.11,AR,ATH,MISC
86,WIPRO,420.00,-15.15,41.0,M-LC,5.77,149463.0,-1482.0,110931.0,0.71,-0.98,74.22,72.51,53.0,-0.01,1.07,4.98,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.58,31.0,H-LC,12.49,272741.0,-73215.0,148071.0,-0.44,-21.16,54.29,21.64,1.0,-0.49,1.95,0.45,X40,ATH,IT
83,VBL,671.64,-21.70,31.0,H-LC,9.55,282187.0,-33655.0,146314.0,-0.15,-10.66,51.85,35.67,5.0,-0.23,2.02,1.72,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-18.03,32.0,H-LC,5.81,206782.0,-44986.0,98201.0,0.60,-17.87,47.49,21.13,27.0,-0.46,1.48,11.23,X40,ATH,PAINTS
30,HCLTECH,1943.91,-4.09,37.0,H-LC,10.27,219298.0,-22626.0,85899.0,0.53,-9.35,39.17,26.15,8.0,-0.26,1.57,3.63,X40,ATH,IT
40,INFY,2275.00,-21.39,37.0,H-LC,8.25,305825.0,-7063.0,178755.0,-0.69,-2.26,58.45,54.87,3.0,-0.04,2.18,4.24,X40,BTT,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,-2.40,46.0,H-LC,1.77,156830.0,-29265.0,76753.0,0.48,-15.73,48.94,25.51,15.0,-0.38,1.12,15.57,AR,ATH,ELECTRICAL
48,LTIM,7230.2,-7.23,42.0,H-LC,1.79,230040.0,-16111.0,95329.0,-0.21,-6.55,41.44,32.18,16.0,-0.17,1.64,27.14,X200,ATH,IT
42,ITC,452.0,-38.77,45.0,H-LC,2.18,197165.0,-2973.0,23857.0,-0.59,-1.49,12.10,10.44,4.0,-0.12,1.41,4.00,X40,NTT,FMCG
51,NESTLEIND,1377.0,-12.04,42.0,H-LC,3.39,270668.0,5242.0,51562.0,-0.83,1.97,19.05,21.40,11.0,0.10,1.93,8.75,XY25,NTT,FMCG
57,RELIANCE,1533.0,-14.79,41.0,H-LC,3.55,213252.0,2886.0,25889.0,-0.12,1.37,12.14,13.68,37.0,0.11,1.52,18.12,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.58,31.0,H-LC,12.49,272741.0,-73215.0,148071.0,-0.44,-21.16,54.29,21.64,1.0,-0.49,1.95,0.45,X40,ATH,IT
83,VBL,671.64,-21.70,31.0,H-LC,9.55,282187.0,-33655.0,146314.0,-0.15,-10.66,51.85,35.67,5.0,-0.23,2.02,1.72,X40N,ATH,FMCG
30,HCLTECH,1943.91,-4.09,37.0,H-LC,10.27,219298.0,-22626.0,85899.0,0.53,-9.35,39.17,26.15,8.0,-0.26,1.57,3.63,X40,ATH,IT
42,ITC,452.00,-38.77,45.0,H-LC,2.18,197165.0,-2973.0,23857.0,-0.59,-1.49,12.10,10.44,4.0,-0.12,1.41,4.00,X40,NTT,FMCG
40,INFY,2275.00,-21.39,37.0,H-LC,8.25,305825.0,-7063.0,178755.0,-0.69,-2.26,58.45,54.87,3.0,-0.04,2.18,4.24,X40,BTT,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-2.40,46.0,H-LC,1.77,156830.0,-29265.0,76753.0,0.48,-15.73,48.94,25.51,15.0,-0.38,1.12,15.57,AR,ATH,ELECTRICAL
42,ITC,452.00,-38.77,45.0,H-LC,2.18,197165.0,-2973.0,23857.0,-0.59,-1.49,12.10,10.44,4.0,-0.12,1.41,4.00,X40,NTT,FMCG
5,ASIANPAINT,3465.66,-18.03,32.0,H-LC,5.81,206782.0,-44986.0,98201.0,0.60,-17.87,47.49,21.13,27.0,-0.46,1.48,11.23,X40,ATH,PAINTS
19,CIPLA,1795.00,-21.49,48.0,H-LC,3.92,209500.0,5000.0,36411.0,1.07,2.45,17.38,20.25,10.0,0.14,1.50,11.39,X40N,BTT,PHARMA
57,RELIANCE,1533.00,-14.79,41.0,H-LC,3.55,213252.0,2886.0,25889.0,-0.12,1.37,12.14,13.68,37.0,0.11,1.52,18.12,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.58,31.0,H-LC,12.49,272741.0,-73215.0,148071.0,-0.44,-21.16,54.29,21.64,1.0,-0.49,1.95,0.45,X40,ATH,IT
40,INFY,2275.00,-21.39,37.0,H-LC,8.25,305825.0,-7063.0,178755.0,-0.69,-2.26,58.45,54.87,3.0,-0.04,2.18,4.24,X40,BTT,IT
42,ITC,452.00,-38.77,45.0,H-LC,2.18,197165.0,-2973.0,23857.0,-0.59,-1.49,12.10,10.44,4.0,-0.12,1.41,4.00,X40,NTT,FMCG
83,VBL,671.64,-21.70,31.0,H-LC,9.55,282187.0,-33655.0,146314.0,-0.15,-10.66,51.85,35.67,5.0,-0.23,2.02,1.72,X40N,ATH,FMCG
1,ABB,7934.00,-40.35,45.0,H-LC,7.65,249240.0,-12379.0,131599.0,-0.13,-4.73,52.80,45.57,7.0,-0.09,1.78,6.05,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,45.0,L-SC,37.50,79930.0,-33619.0,73671.0,0.51,-29.61,92.17,35.27,268.0,-0.46,0.57,95.63,XR,NTT,HOTELS
49,MASFIN,398.61,-19.60,43.0,H-SC,11.59,91380.0,-6600.0,28200.0,-0.73,-6.74,30.86,22.05,152.0,-0.23,0.65,32.83,XR,ATH,FINANCE
50,NATIONALUM,244.55,-42.83,65.0,H-MC,12.87,114575.0,14071.0,15032.0,0.78,14.00,13.12,28.96,79.0,0.94,0.82,52.72,MH,ATH,METALS
78,UJJIVANSFB,60.00,107.70,66.0,H-SC,12.14,131085.0,-11394.0,30910.0,0.66,-8.00,23.58,13.70,163.0,-0.37,0.94,54.72,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-187.50,79.0,M-SC,9.60,151299.0,2079.0,142100.0,-0.87,1.39,93.92,96.62,208.0,0.01,1.08,63.74,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.0,1244.50,69.0,M-SC,18.27,182952.0,1809.0,43652.0,-0.06,1.00,23.86,25.10,235.0,0.04,1.31,46.62,XY24,NTT,HEALTHCARE
69,SURYODAY,240.0,71.05,78.0,H-SC,3.81,157457.0,-21614.0,87341.0,-0.78,-12.07,55.47,36.71,135.0,-0.25,1.12,56.23,XR,NTT,BANKS
61,SAMMAANCAP,326.0,-187.50,79.0,M-SC,9.60,151299.0,2079.0,142100.0,-0.87,1.39,93.92,96.62,208.0,0.01,1.08,63.74,XY25,NTT,FINANCE
78,UJJIVANSFB,60.0,107.70,66.0,H-SC,12.14,131085.0,-11394.0,30910.0,0.66,-8.00,23.58,13.70,163.0,-0.37,0.94,54.72,OX40N,NTT,BANKS
7,ATULAUTO,844.0,3597.30,50.0,M-SC,7.86,166039.0,-31093.0,120926.0,-0.38,-15.77,72.83,45.57,236.0,-0.26,1.19,17.72,XY24,NTT,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.98
1,25,43.07
2,50,74.84


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.26
LC    32.03
MC    23.72
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.01
X40      14.44
X40N     11.25
XR       11.11
XY25     10.38
AR        8.10
OX40N     7.78
X200      1.64
MH        1.63
X5K       1.50
SR        1.17
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.09
H-LC    25.39
H-MC    20.42
M-SC    13.73
M-LC     5.62
M-MC     2.99
L-SC     1.44
L-LC     1.02
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.74,-6.62,41.81
IT,12.43,-20.75,86.58
FINANCE,10.44,-15.42,67.84
MISC,6.94,-18.43,84.28
BANKS,6.46,-10.44,68.54
PAINTS,5.56,-18.56,36.23
ELECTRICAL,5.32,-10.36,49.08
INSURANCE,3.76,-5.29,44.64
AUTO,3.62,-13.74,68.12


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3233480.0,22
XR,1353420.0,14
AR,1196791.0,9
X40,828025.0,10
X40N,629468.0,9
OX40N,559735.0,10
XY25,483186.0,7
SR,258089.0,2
X5K,108051.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3719669.0,29
M-SC,1509345.0,17
H-LC,1265270.0,15
H-MC,1245556.0,15
M-LC,365057.0,4
M-MC,364413.0,2
L-SC,260272.0,3
L-MC,59697.0,1
L-LC,41905.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1224198.0      6
M-SC       XY24       852701.0      7
H-SC       AR         819643.0      5
           XR         781522.0      7
H-MC       XY24       571796.0      4
H-LC       X40        534783.0      5
M-MC       XY24       364413.0      2
H-SC       X40N       310741.0      4
           SR         258089.0      2
           OX40N      254898.0      4
H-LC       X40N       234040.0      3
H-MC       X40        218629.0      4
H-LC       AR         208352.0      2
H-MC       XY25       180712.0      2
L-SC       XR         171895.0      2
M-SC       AR         168796.0      2
M-LC       XY24       146075.0      2
M-SC       XY25       142100.0      1
           OX40N      141538.0      4
           XR         129597.0      2
H-LC       XY25       118469.0      3
M-LC       XR         110931.0      1
           X5K        108051.0      1
H-MC       XR          99778.0      1
H-LC       X200        95329.0      1
L-SC       OX40N       88377.0      1
H-MC       X40N        84687.0      2
           OX40N       74922.0      1
M-SC       X40         74613.0      1
H-LC       XY24        74297.0      1
H-SC       MH          70578.0      1
L-MC       XR          59697.0      1
L-LC       XY25        41905.0      1
H-MC       MH          15032.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
